In [1]:
#抓取百度数据
from bs4 import BeautifulSoup
from selenium import webdriver
import numpy as np
import pandas as pd
import re
import os

from datetime import datetime,timedelta
thedate=datetime.strftime(datetime.today()-timedelta(days=1),"%Y-%m-%d")
thedate2=datetime.strftime(datetime.today()-timedelta(days=1),"%m.%d")

browser = webdriver.Chrome()
url='https://motion.baidu.com/activity/yiqing_predict/main?province=%E5%85%A8%E5%9B%BD'
browser.get(url)

with open(r'./COVIDend/%s.html' % thedate,'w',encoding='utf-8') as f:
    f.write(browser.page_source)

browser.close()

In [6]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
from datetime import datetime,timedelta


city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])
city_popu.head(2)


from bs4 import BeautifulSoup
import lxml
# thedate="2022-12-30"
thedate=datetime.strftime(datetime.today()-timedelta(days=1),"%Y-%m-%d")

soup=BeautifulSoup(open(r'COVIDend\%s.html'  % thedate,'r',encoding='utf-8').read())

trs=soup.find_all('div',{"class":'list-item_1JNax'})

def get_result(tr):
    city=tr.find('div',{'class':'list-item-city_qLfui pc_UIzzf'}).find('div').text.replace('伊犁哈萨克自治州','伊犁')
    progress=tr.find('div',{'class':'rate-text_1PWuo'}).text
    peak=tr.find_all('div',{'class':'flex_3nTxm'})[1].text
    end=tr.find('div',{'class':'flex_3nTxm end_MnHhu'}).text
    return [city.replace('市',''),progress,peak,end]

d=[get_result(tr) for tr in trs]

df=pd.DataFrame(data=d,columns=['city','progress','peak','end'])
df=df.sort_values(by='end')

def covid_progress(s):
    s=s.replace('%','')
    if '回落至' in s:
        s=s.replace('回落至','')
    if '上升至' in s:
        s=s.replace('上升至','')
        s=200-int(s)
    return int(s)    
df['progress_2']=df['progress'].apply(covid_progress)
print(thedate)
df=df[['city','peak','progress_2']]

city99=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='99city')
c1=city99['city'].isin(df['city'])
# c2=df['city'].isin(city99['city'])
city_lack=city99[~c1]
city_lack['progress_2']=0
print(city_lack)
df=pd.concat([df,city_lack])
df

2023-01-15
      city        peak  progress_2
29      上海  2022/12/22           0
1709    东莞  2022/12/22           0
1859    中山  2022/12/20           0
2009    临沂  2022/12/21           0
1139  乌鲁木齐  2022/12/21           0
...    ...         ...         ...
2069    镇江  2022/12/21           0
389     长春  2022/12/17           0
659     长沙  2022/12/21           0
869     青岛  2022/12/21           0
2879    韶关  2022/12/24           0

[97 rows x 3 columns]


,city,peak,progress_2
50,鞍山,2022/12/18,0
38,龙岩,2022/12/27,0
39,丽水,2022/12/24,0
32,本溪,2022/12/18,1
40,舟山,2022/12/26,0
...,...,...,...
2069,镇江,2022/12/21,0
389,长春,2022/12/17,0
659,长沙,2022/12/21,0
869,青岛,2022/12/21,0


In [7]:
df=df.merge(city_popu,left_on='city',right_on='城市')
df['thedate']=thedate
df.to_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
print(thedate)
df.sort_values(by='progress_2')

2023-01-15


,city,peak,progress_2,城市,GDP,人均可支配收入,常住人口,thedate
0,鞍山,2022/12/18,0,鞍山,1739,37980,154,2023-01-15
93,温州,2022/12/22,0,温州,6871,63481,302,2023-01-15
92,清远,2022/12/24,0,清远,1777,33159,174,2023-01-15
91,深圳,2022/12/22,0,深圳,27670,64878,1763,2023-01-15
90,淮安,2022/12/21,0,淮安,4025,40318,299,2023-01-15
...,...,...,...,...,...,...,...,...
31,漳州,2022/12/27,1,漳州,4546,40008,93,2023-01-15
18,伊春,2022/12/22,2,伊春,294,26881,36,2023-01-15
26,儋州,2022/12/25,2,儋州,359,35150,45,2023-01-15
24,三亚,2022/12/23,3,三亚,695,40547,104,2023-01-15


In [12]:
",".join(list(df[df['progress_2']<=1]['city']))

'鞍山,龙岩,丽水,本溪,舟山,七台河,朔州,汕尾,鹤岗,抚顺,潮州,延安,平凉,毕节,威海,锦州,佳木斯,东营,白山,大庆,丽江,葫芦岛,呼伦贝尔,巴彦淖尔,临沧,普洱,北海,漳州,海口,上海,东莞,中山,临沂,乌鲁木齐,佛山,保定,兰州,北京,南京,南充,南宁,南昌,南通,南阳,厦门,台州,合肥,呼和浩特,咸阳,哈尔滨,唐山,嘉兴,大同,大连,天津,太原,宁波,宿迁,常州,广州,廊坊,张家口,徐州,德州,德阳,惠州,成都,扬州,新乡,无锡,昆明,杭州,柳州,桂林,武汉,汕头,江门,沈阳,沧州,泉州,泰安,泰州,洛阳,济南,济宁,淄博,淮安,深圳,清远,温州,湖州,湛江,滁州,潍坊,烟台,珠海,盐城,石家庄,福州,秦皇岛,绍兴,绵阳,肇庆,芜湖,苏州,茂名,衡阳,西宁,西安,贵阳,赣州,连云港,邢台,邯郸,郑州,鄂尔多斯,重庆,金华,银川,镇江,长春,长沙,青岛,韶关'

In [11]:
def covid_status(s):
    s=str(s)
    status=""
    s=s.replace('回落至','').replace('%','')
    if int(s) <=1:
        status='9下降至峰值的0-1%'
    elif 1< int(s) <=10:
        status='8下降至峰值的1-10%'
    elif 10< int(s) <=30:
        status='7下降至峰值的10-30%'
    elif 30< int(s) <=50:
        status='6下降至峰值的30-50%'
    elif 50< int(s) <=80:
        status='5下降至峰值的50-80%'
    elif int(s)>80:
        status='4下降至峰值的80-100%'
    return status

df['status']=df['progress_2'].apply(covid_status)
df['date']=thedate

pv_value=pd.pivot_table(df, values='常住人口', index=['date'],columns=["status"], aggfunc=np.sum)

pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)

import os

final=pv_value
final['0 sum']=1
final=final.fillna(0)
final=final.sort_index(axis=1)
final.to_clipboard(header=None)

final
# pv_value.tail(3)

status,0 sum,8下降至峰值的1-10%,9下降至峰值的0-1%
date,,,
2023-01-15,1,0.005432,0.994568


In [7]:
# wkbk=df.rename(columns={'peak':'搜索峰值日','end':'高峰结束日'})
# title=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='citylist')
# title.merge(wkbk[['city','progress','搜索峰值日','高峰结束日']],how='left',on='city')[['progress','搜索峰值日','高峰结束日']].to_clipboard(index=False)

In [8]:
# #COVID END date

# allpeople=df['常住人口'].sum()
# # allpeople
# df2=df.groupby('end').sum().reset_index()
# df2['人口cumsum']=df2['常住人口'].cumsum()
# df2['人口cumsum_per']=df2['人口cumsum']/allpeople
# df2['sum']=np.nan
# df2[['end','sum','人口cumsum_per']].to_clipboard(index=False,header=None)

In [5]:

from bs4 import BeautifulSoup
import lxml

def getpv(thedate):
    soup=BeautifulSoup(open(r'COVIDend\%s.html'  % thedate,'r',encoding='utf-8').read())

    trs=soup.find_all('div',{"class":'list-item_1JNax'})

    def get_result(tr):
        city=tr.find('div',{'class':'list-item-city_qLfui pc_UIzzf'}).find('div').text.replace('伊犁哈萨克自治州','伊犁')
        progress=tr.find('div',{'class':'rate-text_1PWuo'}).text
        peak=tr.find_all('div',{'class':'flex_3nTxm'})[1].text
        end=tr.find('div',{'class':'flex_3nTxm end_MnHhu'}).text
        return [city.replace('市',''),progress,peak,end]

    d=[get_result(tr) for tr in trs]

    df=pd.DataFrame(data=d,columns=['city','progress','peak','end'])
    df=df.sort_values(by='end')
    df=df.merge(city_popu,left_on='city',right_on='城市')

    def covid_progress(s):
        s=s.replace('%','')
        if '回落至' in s:
            s=s.replace('回落至','')
        if '上升至' in s:
            s=s.replace('上升至','')
            s=200-int(s)
        return int(s)    
    df['progress_2']=df['progress'].apply(covid_progress)

    def covid_status(s):
        status=""
        if '上升至' in s:
            s=s.replace('上升至','').replace('%','')
            if int(s) <=50:
                status='1上升至峰值的0-50%'
            elif 50< int(s) <=80:
                status='2上升至峰值的50-80%'
            elif int(s)>80:
                status='3上升至峰值的80-100%'
        if '回落至' in s:
            s=s.replace('回落至','').replace('%','')
            if int(s) <=1:
                status='9下降至峰值的0-1%'
            elif 1< int(s) <=10:
                status='8下降至峰值的1-10%'
            elif 10< int(s) <=30:
                status='7下降至峰值的10-30%'
            elif 30< int(s) <=50:
                status='6下降至峰值的30-50%'
            elif 50< int(s) <=80:
                status='5下降至峰值的50-80%'
            elif int(s)>80:
                status='4下降至峰值的80-100%'
        return status

    df['status']=df['progress'].apply(covid_status)
    df['date']=thedate



    pv_value=pd.pivot_table(df, values='常住人口', index=['date'],columns=["status"], aggfunc=np.sum)

    pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)
    return pv_value

import os
final=pd.DataFrame()
for f in os.listdir(r'COVIDend'):
    thedate=f.replace('.html','')
    final=pd.concat([final,getpv(thedate)])

final['0 sum']=1
final=final.fillna(0)
final=final.sort_index(axis=1)
final.tail(22).to_clipboard(header=None)

final.tail(22)
# pv_value.tail(3)

status,0 sum,1上升至峰值的0-50%,2上升至峰值的50-80%,3上升至峰值的80-100%,4下降至峰值的80-100%,5下降至峰值的50-80%,6下降至峰值的30-50%,7下降至峰值的10-30%,8下降至峰值的1-10%,9下降至峰值的0-1%
date,,,,,,,,,,
2022-12-18,1,0.035309,0.192308,0.373902,0.359497,0.038985,0.000000,0.000000,0.000000,0.000000
2022-12-19,1,0.035046,0.190876,0.371118,0.209385,0.182808,0.005858,0.004909,0.000000,0.000000
2022-12-20,1,0.002557,0.039172,0.314885,0.277620,0.317044,0.041094,0.007627,0.000000,0.000000
2022-12-21,1,0.000000,0.027279,0.225210,0.265670,0.413453,0.060755,0.007633,0.000000,0.000000
2022-12-22,1,0.000000,0.008640,0.136034,0.274860,0.431649,0.139434,0.006636,0.002747,0.000000
2022-12-23,1,0.000000,0.012012,0.119395,0.285921,0.286305,0.162700,0.130934,0.002733,0.000000
2022-12-24,1,0.000000,0.002805,0.108921,0.177414,0.333798,0.166179,0.202942,0.007940,0.000000
2022-12-25,1,0.000000,0.000000,0.047416,0.110329,0.358914,0.204460,0.265929,0.012952,0.000000
2022-12-26,1,0.000000,0.000000,0.017671,0.073696,0.276406,0.282346,0.303252,0.046629,0.000000
